# 模型微调
冻结某些层，通过特定数据，调整模型的某些层。
需要数据集，对数据集进行预处理。 常见的方法有 截断和填充
对于自然语言，需要通过分词器（token）将自然语言分割，并进行embedding，转为向量（固定长度） 每个单词，通过分词器就会映射到 词表对应的一个 tokenid 上
进行模型的微调
对微调后的模型进行 基准测试
微调后模型的保存




- 数据处理
- 加载数据
- 数据预处理 - 截断和填充



In [8]:
from datasets import load_dataset
from transformers import AutoModel, AutoTokenizer,pipeline

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

print(type(tokenizer))

dataset =  load_dataset("yelp_review_full")

print(dataset)

dataset["train"][111]

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True,padding="max_length")

tokenized_datasets = dataset.map(tokenize_function,batched=True)

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))


print(small_train_dataset)



<class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>
DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})
Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})


In [3]:
from transformers import AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=5)

trainArgs = TrainingArguments(output_dir= "/mnt/e/new_volume/models",
                                 per_device_train_batch_size=16,   ## 训练的批次大小，越大的批次，加载的数据越多
                                  num_train_epochs=5, ## 整个数据集的遍历次数
                                  logging_steps=100) ## 每隔一百次记录一次训练日志

trainer = Trainer(model=model,args=trainArgs,  ## 模型参数
                  train_dataset=small_train_dataset,   ## 训练集数据
                    eval_dataset=small_eval_dataset)   ## 验证集数据
trainer.train()


## 验证

small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(100))
trainer.evaluate(small_test_dataset)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
100,1.170700
200,0.645100
300,0.335100


{'eval_loss': 1.2619549036026,
 'eval_runtime': 1.4347,
 'eval_samples_per_second': 69.7,
 'eval_steps_per_second': 9.061,
 'epoch': 5.0}

In [10]:
! nvidia-smi

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: line 1: nvidia-smi: command not found


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## 问答任务的训练


In [5]:
## 加载数据集
from datasets import load_dataset

squad_data = load_dataset("squad")

## 每个数据集都有自己的格式和字段 ，还有训练集和测试集


print(squad_data)


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})


In [7]:
## 预处理数据

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

print(tokenizer.model_max_length)

token = tokenizer("北京在哪里？","北京在中国")

print(token)


512
{'input_ids': [101, 1781, 1755, 100, 100, 1962, 1994, 102, 1781, 1755, 100, 1746, 1799, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
